## Load execution times

In [5]:
import pandas as pd
from etl import files
import tempfile
import json
from pathlib import Path


with tempfile.NamedTemporaryFile(delete=False) as temp_file:
    files.download_file_from_server(Path(temp_file.name), f"owid@etl-prod-2:~/etl/.execution_time.json")
    temp_file_path = temp_file.name

with open(temp_file_path, 'r') as f:
    data = json.load(f)

data = pd.Series(data)
data.shape

2024-12-19 10:33:58 [info     ] file.downloaded                path=PosixPath('/var/folders/0s/2yqr44dj44zcmyzdrf8fvxyc0000gn/T/tmpxyijvoya') target=owid@etl-prod-2:~/etl/.execution_time.json


(4659,)

## Longest steps

In [14]:
data.sort_values(ascending=False).head(20).to_frame('t').style.format('{:.0f}').bar(color='orange')

,t
grapher://grapher/who/2021-07-01/ghe,3547
data-private://garden/ihme_gbd/2024-05-20/gbd_prevalence,1625
grapher://grapher/ihme_gbd/2019/gbd_prevalence,1328
data://garden/un/2022-07-11/un_wpp,1237
data://garden/faostat/2024-03-14/faostat_fbsc,1224
data://meadow/un/2022-07-11/un_wpp,1191
data-private://garden/ihme_gbd/2024-05-20/gbd_cause,1090
grapher://grapher/ihme_gbd/2019/gbd_cause,970
grapher://grapher/ihme_gbd/2024-05-20/gbd_risk,934
grapher://grapher/ihme_gbd/2024-05-20/gbd_prevalence,850


## Time per step type & channel

In [37]:
df = data.to_frame('t').reset_index().rename(columns={'index': 'step'})
df['channel'] = df.step.map(lambda s: '/'.join(s.split('/')[:3])).str.replace('-private', '')
df = df[~df['channel'].str.contains('snapshot://')]
df.groupby('channel').t.sum().sort_values(ascending=False).to_frame('t').style.format('{:.0f}').bar(color='orange')

,t
channel,
grapher://grapher,21617
data://garden,19388
data://meadow,14428
data://grapher,4956
backport://backport,1518
data://explorers,1011
data://open_numbers,38
export://multidim,15
data://external,12
